### Problem 9

In [26]:
#### import yfinance as yf
import pandas as pd
from datetime import datetime

today_date = datetime.today().date()
frac = get_fraction_of_year_remaining(today_date)


PV = []
latestPrice = None

print("ATTENTION: This program assumes it is currently 2024. Let's find the Intrinsic Value of a Stock.\n")

while True:
    try:
        k = float(input("What is the Discount Rate (%): ")) / 100
        break
    except ValueError:
        print("\nPlease enter a valid number for the discount rate.")

while latestPrice is None:
    ticker = getTicker()
    latestPrice = getStockPrice(ticker)
    if latestPrice is None:
            print("I couldn't find that stock! Please try again with a valid stock!\n")

year1, DIV = getDiscrete(k)
getGrowth(k, year1, DIV)
getRecommendation(ticker, latestPrice, k, frac)

ATTENTION: This program assumes it is currently 2024. Let's find the Intrinsic Value of a Stock.



What is the Discount Rate (%):  4.8



What stock are you interested in valuing?


Ticker:  wmt



In what year do the dividends stop being discrete? Hit 'enter' if there are no discrete dividends.


Year:  2028
What is the cash flow for 2024:  0.83
What is the cash flow for 2025:  0.94
What is the cash flow for 2026:  1.08
What is the cash flow for 2027:  1.18
What is the cash flow for 2028:  1.23



How many dividend growth periods? Hit 'enter' if there are none.


Number of Periods:  1

Dividend growth rate for this period (%):  3



Based on the given information, wmt has an intrinsic value (based on DDM) of: $61.92.
wmt's value of $61.92 is less than the latest price of $67.59. I recommend SELLING wmt.

Disclamer: Not a financial advisor.


In [25]:
def getTicker():
    print("\nWhat stock are you interested in valuing?")
    ticker = input("Ticker: ")
    return(ticker)

In [24]:
def getStockPrice(ticker):
    stock = yf.Ticker(ticker)
    stock_info = stock.history(period='1d')
    if not stock_info.empty:
        latest_price = stock_info['Close'].iloc[-1]
        return latest_price
    else:
        return None

In [23]:
def getDiscrete(k):
    print("\nIn what year do the dividends stop being discrete? Hit 'enter' if there are no discrete dividends.")
    global PV
    
    while True:
        try:
            year = input("Year: ")
            if year == "":
                break
            year = int(year)
            if year < 2024:
                print("\nPlease enter a valid year.")
            else:
                break
        except ValueError:
            print("\nPlease enter a valid number.")
            
    if year == "":
        PV.append(0)
        print("\nNo discrete dividend periods.")
        return(2024, 0)
    else:
        for i in range(0, (year - 2024) + 1):
            while True:
                try:
                    CF = float(input(f"What is the cash flow for {2024 + i}: "))
                    break
                except ValueError:
                    print("\nPlease enter a valid number for the cash flow.")
    
            PV.append(CF / (1 + k) ** i)
        return (year, CF)

In [22]:
def getGrowth(k, year1, DIV):
    print("\nHow many dividend growth periods? Hit 'enter' if there are none.")
    global PV

    while True: 
        try: 
            periods = input('Number of Periods: ')
            if periods == "":
                break 
            periods = int(periods)
            if periods < 0:
                print("\nPlease enter a valid number of periods.")
            else: 
                break
        except ValueError:
            print("\nPlease enter a valid number of periods.")

    if periods == "":
        PV.append(0)
        print("\nNo dividend growth periods.")
    else: 
        for i in range(1, periods+1):
            while True:
                try: 
                    g = float(input("\nDividend growth rate for this period (%): "))
                    if g < 0:
                        print("\nPlease enter a valid number.")
                    else:
                        break 
                except ValueError: 
                    print("\nPlease enter a valid number.")
            g = g / 100
            if i < periods:
                while True:
                    try:
                        year2 = int(input("\nIn what year does this growth period end? "))
                        if year2 < year1:
                            print("\nPlease enter a valid year.")
                        else:
                            break
                    except ValueError:
                        print("\nPlease enter a valid number.")
                
                PV.append((1 / (1+k)**(year1 - 2024)) * (((DIV * (1 + g)) / (k - g)) * (1 - ((1 + g) / (1 + k))**(year2 - year1))))
                DIV = DIV * (1+g)**(year2 - year1)
                year1 = year2
            else: 
                PV.append((1 / (1+k)**(year1 - 2024))*(DIV * (1 + g) / (k - g)))
            
                    

In [21]:
def getRecommendation(ticker, latestPrice, k, year_frac):
    global PV
    value = sum(PV) / (1+k)**(year_frac)
    print(f"\nBased on the given information, {ticker} has an intrinsic value (based on DDM) of: ${value:,.2f}.")
    
    if int(value) == int(latestPrice):
        print(f"{ticker}'s value of ${value:,.2f} is roughly equal to the latest price of ${latestPrice:,.2f}. I recommend HOLDING {ticker}.")
    
    elif value > latestPrice:
         print(f"{ticker}'s value of ${value:,.2f} is more than the latest price of ${latestPrice:,.2f}. I recommend BUYING {ticker}.")
        
    elif value < latestPrice:
        print(f"{ticker}'s value of ${value:,.2f} is less than the latest price of ${latestPrice:,.2f}. I recommend SELLING {ticker}.")
    print("\nDisclamer: Not a financial advisor.")

In [20]:
def get_fraction_of_year_remaining(today_date):
    # Ensure today_date is a datetime object
    today_date = datetime.combine(today_date, datetime.min.time())
    
    # Get the last day of the year
    end_of_year = datetime(today_date.year, 12, 31)
    
    # Calculate the number of days remaining in the year
    days_remaining = (end_of_year - today_date).days
    
    # Calculate the total number of days in the year
    is_leap_year = today_date.year % 4 == 0 and (today_date.year % 100 != 0 or today_date.year % 400 == 0)
    total_days_in_year = 366 if is_leap_year else 365
    
    # Calculate the fraction of the year remaining
    fraction_of_year_remaining = days_remaining / total_days_in_year
    return fraction_of_year_remaining